
# Backtest Strategi dengan Data Langsung dari OKX

Notebook ini memuat data harga langsung dari publik API OKX sehingga Anda tidak perlu lagi menyiapkan file CSV secara terpisah. Cukup atur parameter di sel konfigurasi dan jalankan seluruh notebook untuk menarik data, menjalankan strategi, dan melihat hasil backtest.



## Cara menggunakan
1. Jalankan sel impor di bawah ini.
2. Ubah parameter di bagian **Konfigurasi dataset & strategy** sesuai kebutuhan (instrumen, timeframe, dan rentang waktu).
3. Jalankan sel-sel berikutnya untuk menarik data dari OKX dan menjalankan strategi yang dipilih.


In [1]:

from __future__ import annotations

import json
import sys
import time
import socket
from pathlib import Path
from typing import Dict, List, Tuple
from urllib import error, parse, request

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython import get_ipython
from IPython.display import display

PROJECT_ROOT = Path('..').resolve()
for path in (PROJECT_ROOT, PROJECT_ROOT / 'src'):
    if str(path) not in sys.path:
        sys.path.append(str(path))

_ip = get_ipython()
if _ip is not None:
    try:
        _ip.run_line_magic('matplotlib', 'inline')
    except AttributeError:
        plt.switch_backend('Agg')
else:
    plt.switch_backend('Agg')

plt.style.use('seaborn-v0_8-darkgrid')

from src.strategy_backtest import SignalBacktester, get_strategy, list_strategies
from src.strategy_backtest.utils import load_strategy_csv, sanitise_columns


In [2]:

# Konfigurasi dataset & strategy
ASSET_SYMBOL = 'ETHUSDT'
OKX_INSTRUMENT = 'ETH-USDT-SWAP'  # Format instId, contoh lain: BTC-USDT-SWAP, ETH-USDT
TIMEFRAME = '1H'  # Lihat dokumentasi OKX untuk opsi lain: 1m, 5m, 15m, 1H, 4H, 1D, dst.
START_TIME = '2023-01-01'  # Gunakan None untuk mengambil seluruh riwayat tersedia
END_TIME = '2025-11-01'    # Gunakan None untuk tanggal saat ini
REQUEST_LIMIT = 200        # Maksimum candle per permintaan (OKX mengizinkan hingga 300)
REQUEST_SLEEP = 0.2        # Jeda antar permintaan agar tidak terkena rate-limit

TIME_COLUMN = 'time'
PRICE_COLUMN = 'close'

# Ganti nama strategy sesuai file di `src/strategy_backtest/strategies/`
STRATEGY_NAME = 'vwap'
# Opsional: override parameter default strategy
STRATEGY_PARAMS = {}

LOCAL_DATA_PATH = PROJECT_ROOT / 'data' / 'sample_1h_data.csv'


In [3]:
OKX_CANDLES_URL = 'https://www.okx.com/api/v5/market/candles'


def _to_millis(value):
    if value in (None, ''):
        return None
    ts = pd.Timestamp(value, tz='UTC')
    return int(ts.timestamp() * 1000)


def _load_local_dataset(path: Path) -> Tuple[pd.DataFrame, Dict[str, str]]:
    df, mapping = load_strategy_csv(path)
    df = df.sort_index()
    return df, mapping


def fetch_okx_ohlcv(
    inst_id: str,
    bar: str,
    start=None,
    end=None,
    limit: int = 200,
    pause: float = 0.2,
    fallback_path: Path | str | None = None,
    request_timeout: float = 10.0,
) -> Tuple[pd.DataFrame, Dict[str, str]]:
    start_ms = _to_millis(start)
    end_ms = _to_millis(end)
    if end_ms is None:
        end_ms = int(pd.Timestamp.utcnow().timestamp() * 1000)

    fallback_path = Path(fallback_path) if fallback_path is not None else None

    params = {'instId': inst_id, 'bar': bar, 'limit': str(limit)}
    all_rows: List[List[str]] = []
    cursor = end_ms

    def _use_fallback(reason: str, exc: Exception | None = None):
        if fallback_path is None:
            if exc is not None:
                raise RuntimeError(reason) from exc
            raise RuntimeError(reason)
        print(f"⚠️ {reason}. Menggunakan dataset lokal: {fallback_path}")
        return _load_local_dataset(fallback_path)

    while True:
        query = params | {'before': str(cursor)}
        url = f"{OKX_CANDLES_URL}?{parse.urlencode(query)}"
        try:
            with request.urlopen(url, timeout=request_timeout) as resp:
                payload = json.loads(resp.read().decode('utf-8'))
        except (error.URLError, TimeoutError, socket.timeout) as exc:
            return _use_fallback(f"Gagal terhubung ke OKX ({exc})", exc)

        if payload.get('code') != '0':
            message = f"Gagal mengambil data OKX: {payload.get('msg')} (code={payload.get('code')})"
            return _use_fallback(message)

        rows = payload.get('data', [])
        if not rows:
            break

        all_rows.extend(rows)
        earliest = min(int(row[0]) for row in rows)
        if len(rows) < int(params['limit']):
            break
        if start_ms is not None and earliest <= start_ms:
            break
        if earliest == cursor:
            earliest -= 1
        cursor = earliest
        time.sleep(max(pause, 0))

    if not all_rows:
        return _use_fallback('Tidak ada data OHLCV yang diterima dari OKX')

    df = pd.DataFrame(
        all_rows,
        columns=['ts', 'o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote', 'confirm'],
    )
    df = df.drop_duplicates(subset='ts')
    df['ts'] = pd.to_numeric(df['ts'], errors='coerce')
    df = df.dropna(subset=['ts'])
    df = df.sort_values('ts')

    if start_ms is not None:
        df = df[df['ts'] >= start_ms]
    if end_ms is not None:
        df = df[df['ts'] <= end_ms]

    numeric_cols = ['o', 'h', 'l', 'c', 'vol', 'volCcy', 'volCcyQuote']
    for column in numeric_cols:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    df['confirm'] = df['confirm'].astype(int)

    df['time'] = pd.to_datetime(df['ts'], unit='ms', utc=True).tz_convert(None)
    df = df.drop(columns=['ts'])
    df = df.dropna(subset=['time'])

    rename_map = {
        'o': 'open',
        'h': 'high',
        'l': 'low',
        'c': 'close',
        'vol': 'volume',
        'volCcy': 'volume_ccy',
        'volCcyQuote': 'volume_quote',
        'confirm': 'confirm',
    }
    df = df.rename(columns=rename_map)
    raw_mapping: Dict[str, str] = {'time': 'ts'} | {new: original for original, new in rename_map.items()}
    sanitised_cols, _ = sanitise_columns(df.columns)
    column_mapping = {alias: raw_mapping[col_name] for alias, col_name in zip(sanitised_cols, df.columns)}
    df.columns = sanitised_cols
    df = df.set_index('time').sort_index()

    return df, column_mapping


data, column_mapping = fetch_okx_ohlcv(
    inst_id=OKX_INSTRUMENT,
    bar=TIMEFRAME,
    start=START_TIME,
    end=END_TIME,
    limit=REQUEST_LIMIT,
    pause=REQUEST_SLEEP,
    fallback_path=LOCAL_DATA_PATH,
)

print(f'Dataset berisi {len(data):,} bar dengan {len(data.columns)} kolom.')
if not data.empty:
    print(f"Rentang waktu: {data.index.min()} -> {data.index.max()}")
print('Contoh mapping kolom (sanitised -> OKX):')
for alias, original in list(column_mapping.items())[:10]:
    print(f'  {alias} -> {original}')

display(data.head())


⚠️ Gagal terhubung ke OKX (HTTP Error 403: Forbidden). Menggunakan dataset lokal: C:\Users\jefri\backtest-indicator\data\sample_1h_data.csv
Dataset berisi 720 bar dengan 5 kolom.
Rentang waktu: 2023-01-01 00:00:00 -> 2023-01-30 23:00:00
Contoh mapping kolom (sanitised -> OKX):
  time -> time
  open -> open
  high -> high
  low -> low
  close -> close
  volume -> volume


,open,high,low,close,volume
time,,,,,
2023-01-01 00:00:00,20034.505517,20038.934366,20031.957019,20034.028145,5677.546645
2023-01-01 01:00:00,20097.179713,20097.507459,20082.578983,20108.425611,8387.724448
2023-01-01 02:00:00,19991.049473,19994.888545,19979.649316,19997.957481,6945.530094
2023-01-01 03:00:00,19957.315678,19960.171992,19918.448390,19954.852657,1892.921574
2023-01-01 04:00:00,19934.302284,19952.923133,19926.550381,19938.669364,3564.471175


In [4]:

available = list_strategies()
print('Strategi tersedia:', ', '.join(available))
strategy = get_strategy(STRATEGY_NAME, **STRATEGY_PARAMS)
print('Deskripsi strategi:')
print(f"- Nama: {strategy.metadata.name}")
print(f"- Deskripsi: {strategy.metadata.description}")
print(f"- Entry: {strategy.metadata.entry}")
print(f"- Exit: {strategy.metadata.exit}")
print('Parameter default:')
for key, value in strategy.metadata.parameters.items():
    print(f'  {key}: {value}')
if not strategy.metadata.parameters:
    print('  (tidak ada parameter default eksplisit)')
print('Parameter aktif:')
for key, value in strategy.params.items():
    print(f'  {key}: {value}')
if not strategy.params:
    print('  (menggunakan nilai default)')


Strategi tersedia: ema112_atr, vwap
Deskripsi strategi:
- Nama: vwap
- Deskripsi: Strategi VWAP yang mencari pantulan counter-trend di bawah VWAP dan fade rally di atas VWAP saat tren turun.
- Entry: Long ketika harga pertama kali menyelam di bawah VWAP dan ada konfirmasi bullish (wick bawah, volume naik, RSI > 30) untuk potensi reversion. Short ketika harga memantul di atas VWAP dalam tren turun dengan candle rejection, volume melemah, RSI < 70 dan MACD histogram melemah.
- Exit: Keluar utama di level VWAP sebagai target reversion. Stop-loss menggunakan ATR. Jika target VWAP tercapai, posisi ditutup penuh.
Parameter default:
  rsi_window: 14
  volume_ma_window: 20
  volume_spike_ratio: 1.1
  volume_fade_ratio: 0.9
  atr_window: 14
  atr_stop_multiplier: 1.5
  min_reversion_atr: 0.5
  session_frequency: 1D
  macd_fast: 12
  macd_slow: 26
  macd_signal: 9
  wick_ratio_threshold: 0.6
  allow_counter_trend_long: True
  allow_trend_short: True
Parameter aktif:
  rsi_window: 14
  volume_ma_

In [5]:

signals = strategy.generate_signals(data)
print('Kolom sinyal:', list(signals.columns))
display(signals.head())

backtester = SignalBacktester(data=data, price_column=PRICE_COLUMN)
outputs = backtester.run(signals)

print('Metrik performa:')
for key, value in outputs.metrics.items():
    if isinstance(value, (int, float, np.floating)):
        print(f'- {key}: {value:.4f}')
    else:
        print(f'- {key}: {value}')

print('Ringkasan trade:')
for key, value in outputs.trade_summary.items():
    print(f'- {key}: {value}')

display(outputs.trades.head())


Kolom sinyal: ['long_entry', 'long_exit', 'short_entry', 'short_exit', 'vwap', 'rsi', 'macd', 'macd_signal', 'macd_hist', 'atr', 'volume_ma', 'active_entry_price', 'stop_level', 'target_level', 'exit_flag', 'position']


,long_entry,long_exit,short_entry,short_exit,vwap,rsi,macd,macd_signal,macd_hist,atr,volume_ma,active_entry_price,stop_level,target_level,exit_flag,position
time,,,,,,,,,,,,,,,,
2023-01-01 00:00:00,False,False,False,False,20034.973177,NaN,0.000000,0.000000,0.000000,6.977348,5677.546645,NaN,NaN,20034.973177,<NA>,flat
2023-01-01 01:00:00,False,False,False,False,20071.467876,NaN,5.934841,1.186968,4.747872,11.013203,7032.635546,NaN,NaN,20071.467876,<NA>,flat
2023-01-01 02:00:00,False,False,False,False,20044.812042,89.749023,1.704738,1.290522,0.414216,19.424852,7003.600396,NaN,NaN,20044.812042,<NA>,flat
2023-01-01 03:00:00,False,False,False,False,20036.520820,86.042622,-5.067433,0.018931,-5.086364,23.716583,5725.930690,NaN,NaN,20036.520820,<NA>,flat
2023-01-01 04:00:00,False,False,False,False,20023.438997,84.629531,-11.606491,-2.306153,-9.300338,24.044133,5293.638787,NaN,NaN,20023.438997,<NA>,flat


Metrik performa:
- total_return: -0.0120
- cagr: -0.0042
- sharpe_ratio: -0.9934
- annualised_vol: 0.0042
- max_drawdown: -0.0120
- avg_drawdown_duration: 623.0000
Ringkasan trade:
- total_trades: 2
- long_trades: 1
- short_trades: 1
- win_rate: 0.0
- avg_pnl_pct: -0.006019007881639116
- median_bars: 2.5


,trade_id,direction,entry_time,exit_time,entry_price,exit_price,pnl_pct,pnl_currency,bars_held,exit_reason,...,exit_macd,exit_macd_signal,exit_macd_hist,exit_atr,exit_volume_ma,exit_active_entry_price,exit_stop_level,exit_target_level,exit_exit_flag,exit_position
0,1,Short,2023-01-05,2023-01-05 04:00:00,20086.344237,20212.587686,-0.006285,-126.243449,4,short_exit_signal:stop,...,12.830884,17.200983,-4.370099,61.410981,5169.634925,20086.344237,20189.154379,20115.158215,stop,short
1,2,Long,2023-01-14,2023-01-14 01:00:00,21167.135989,21045.361939,-0.005753,-121.774050,1,long_exit_signal:stop,...,40.762859,29.421124,11.341734,69.805018,6241.511170,21167.135989,21072.639264,21168.461886,stop,long


In [6]:

def _as_bool(series: pd.Series | None) -> pd.Series:
    if series is None:
        return pd.Series(False, index=data.index)
    aligned = series.reindex(data.index)
    return aligned.fillna(False).astype(bool)

close_prices = data[PRICE_COLUMN]
long_entries = _as_bool(signals.get('long_entry'))
long_exits = _as_bool(signals.get('long_exit'))
short_entries = _as_bool(signals.get('short_entry'))
short_exits = _as_bool(signals.get('short_exit'))

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(close_prices.index, close_prices, label='Close', color='black', linewidth=1.2)

if long_entries.any():
    ax.scatter(close_prices.index[long_entries], close_prices[long_entries], marker='^', color='green', label='Long Entry', zorder=5)
if short_entries.any():
    ax.scatter(close_prices.index[short_entries], close_prices[short_entries], marker='v', color='red', label='Short Entry', zorder=5)
if long_exits.any():
    ax.scatter(close_prices.index[long_exits], close_prices[long_exits], marker='v', color='tab:blue', label='Long Exit', zorder=6)
if short_exits.any():
    ax.scatter(close_prices.index[short_exits], close_prices[short_exits], marker='^', color='tab:orange', label='Short Exit', zorder=6)

ax.set_title(f'{ASSET_SYMBOL} Close dengan Sinyal {STRATEGY_NAME}')
ax.set_ylabel('Harga')
ax.legend(loc='upper left', ncol=2)
fig.tight_layout()
display(fig)
plt.close(fig)

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(outputs.results.index, outputs.results['equity_curve'], color='C4', label='Equity Curve')
ax.set_title('Equity Curve Strategi')
ax.set_ylabel('Notional')
ax.legend()
fig.tight_layout()
display(fig)
plt.close(fig)


<Figure size 1400x600 with 1 Axes>

<Figure size 1400x400 with 1 Axes>

In [7]:
display(outputs.results.head())


,close,asset_return,position,strategy_return,equity_curve,drawdown,cumulative_pnl
time,,,,,,,
2023-01-01 00:00:00,20034.028145,0.000000,0.0,0.0,1.0,0.0,0.0
2023-01-01 01:00:00,20108.425611,0.003714,0.0,0.0,1.0,0.0,0.0
2023-01-01 02:00:00,19997.957481,-0.005494,0.0,-0.0,1.0,0.0,0.0
2023-01-01 03:00:00,19954.852657,-0.002155,0.0,-0.0,1.0,0.0,0.0
2023-01-01 04:00:00,19938.669364,-0.000811,0.0,-0.0,1.0,0.0,0.0
